In [1]:
import tensorflow as tf
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
 
# setting path
sys.path.append('../')

In [4]:
from bbox_utils import *
from classification_loss import *

In [5]:
h = tf.keras.losses.Huber(delta=float('inf'), reduction=tf.keras.losses.Reduction.NONE)

In [6]:
actual_deltas =  tf.constant(
    [
        [
            [-0.1, -0.05, 0, 0.091, 0.],
            [0.1, 0.0022, 0.25, 0.14, 1.]
        ],
        [
            [0.1, -0.05, 0, 0.091, 0.],
            [0.1, 0.0022, 0.25, 0.14, 1.]
        ],
        [
            [0.1, -0.05, 0, 0.091, 1.],
            [0.1, 0.0022, 0.25, 0.14, 1.]
        ]
    ]
)

pred_deltas = tf.constant(
    [
        [
            [-0.05, 0.05, 0.1, 0.2],
            [0.1, 0.2, -0.1, 0.02]
        ],
        [
            [-0.05, 0.05, 0.1, 0.2],
            [0.1, 0.2, -0.1, 0.02]
        ],
        [
            [-0.05, 0.05, 0.1, 0.2],
            [0.1, 0.2, -0.1, 0.02]
        ]
    ]
)

loss = h(actual_deltas[:, :, :4], pred_deltas)


2022-09-02 18:37:55.544231: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
actual_deltas.shape

TensorShape([3, 2, 5])

In [8]:
loss

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.00429763, 0.0220031 ],
       [0.00679763, 0.0220031 ],
       [0.00679763, 0.0220031 ]], dtype=float32)>

In [9]:
loss = loss * actual_deltas[:, :, 4]

In [10]:
loss

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.        , 0.0220031 ],
       [0.        , 0.0220031 ],
       [0.00679763, 0.0220031 ]], dtype=float32)>

In [11]:
tf.reduce_sum(loss, axis=-1)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.0220031 , 0.0220031 , 0.02880073], dtype=float32)>

In [12]:
pos = tf.constant([[0.,1.]])

In [13]:
tf.expand_dims(pos, -1)

<tf.Tensor: shape=(1, 2, 1), dtype=float32, numpy=
array([[[0.],
        [1.]]], dtype=float32)>

In [14]:
x = tf.constant([
    [
        [-0.05, 0.05, 0.1, 0.2],
        [0.1, 0.2, -0.1, 0.02]
    ]
])

In [15]:
x

<tf.Tensor: shape=(1, 2, 4), dtype=float32, numpy=
array([[[-0.05,  0.05,  0.1 ,  0.2 ],
        [ 0.1 ,  0.2 , -0.1 ,  0.02]]], dtype=float32)>

In [16]:
y = tf.concat([ x, tf.expand_dims(pos, -1) ], axis = 2)

In [17]:
y

<tf.Tensor: shape=(1, 2, 5), dtype=float32, numpy=
array([[[-0.05,  0.05,  0.1 ,  0.2 ,  0.  ],
        [ 0.1 ,  0.2 , -0.1 ,  0.02,  1.  ]]], dtype=float32)>

In [18]:
y[:, :, :4]

<tf.Tensor: shape=(1, 2, 4), dtype=float32, numpy=
array([[[-0.05,  0.05,  0.1 ,  0.2 ],
        [ 0.1 ,  0.2 , -0.1 ,  0.02]]], dtype=float32)>

In [19]:
feature_map_shapes = [38, 19, 10, 5, 3, 1]
aspect_ratios = [
    [1, 2, 0.5],
    [1, 2, 3, 0.5, 0.333],
    [1, 2, 3, 0.5, 0.333],
    [1, 2, 3, 0.5, 0.333],
    [1, 2, 0.5],
    [1, 2, 0.5]
]
number_of_feature_maps = 6

In [20]:
prior_boxes = generate_default_boxes(feature_map_shapes, number_of_feature_maps, aspect_ratios)
boxes = convert_to_box_form(prior_boxes)

In [21]:
gt_boxes = tf.constant([[
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]
    ],
    [
        [0., 0., 0.2, 0.2],
        [0.8, 0.8, 1.0, 0.9],
    ]
])

gt_labels = tf.constant([
            [0, 0],
            [1, 2]
])

threshold = 0.5

In [22]:
gt_boxes_map_offset_suppressed_with_pos_cond, gt_labels_one_hot_encoded = match_priors_with_gt(prior_boxes, boxes, gt_boxes, gt_labels, number_of_labels = 21, threshold = 0.5)

In [23]:
gt_boxes_map_offset_suppressed_with_pos_cond

<tf.Tensor: shape=(2, 8732, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [24]:
# gt_labels_one_hot_encoded = tf.concat( [ gt_labels_one_hot_encoded, gt_labels_one_hot_encoded ], axis = 0)
# gt_boxes_map_offset_suppressed_with_pos_cond = tf.concat( [ gt_boxes_map_offset_suppressed_with_pos_cond, gt_boxes_map_offset_suppressed_with_pos_cond ], axis = 0)

In [25]:
pred_labels = tf.random.uniform(shape = (1, 8732, 21))

In [26]:
gt_boxes_map_offset_suppressed_with_pos_cond.shape

TensorShape([2, 8732, 5])

In [27]:
pred_labels = pred_labels / tf.math.reduce_sum(
    pred_labels, axis = 1
)

In [28]:
pred_labels = tf.concat( [ pred_labels, pred_labels ], axis = 0)

In [29]:
pred_delta = gt_boxes_map_offset_suppressed_with_pos_cond[:, :, :4] + tf.random.normal(shape = (2, 8732, 4) )

In [30]:
pred_delta.shape

TensorShape([2, 8732, 4])

In [31]:
huber_loss = tf.keras.losses.Huber(
    reduction=tf.keras.losses.Reduction.NONE
)

huber_loss(gt_boxes_map_offset_suppressed_with_pos_cond[:, :, :4], pred_delta)

<tf.Tensor: shape=(2, 8732), dtype=float32, numpy=
array([[0.85173035, 0.46946216, 0.6843491 , ..., 0.7290499 , 0.17973682,
        0.10442332],
       [0.6552733 , 0.04071367, 0.22372013, ..., 0.14258406, 0.28669974,
        0.0519913 ]], dtype=float32)>

In [32]:
huber_loss(gt_boxes_map_offset_suppressed_with_pos_cond[:, :, :4], pred_delta)

<tf.Tensor: shape=(2, 8732), dtype=float32, numpy=
array([[0.85173035, 0.46946216, 0.6843491 , ..., 0.7290499 , 0.17973682,
        0.10442332],
       [0.6552733 , 0.04071367, 0.22372013, ..., 0.14258406, 0.28669974,
        0.0519913 ]], dtype=float32)>

In [33]:
gt_boxes_map_offset_suppressed_with_pos_cond

<tf.Tensor: shape=(2, 8732, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [34]:
# huber loss over all preds
huber_loss = tf.keras.losses.Huber(
    reduction=tf.keras.losses.Reduction.NONE
)
localization_loss_for_all_priors = huber_loss(gt_boxes_map_offset_suppressed_with_pos_cond[:, :, :4], pred_delta)

# localization loss is only for default prior boxes with IOU > 0.5 over ground truth boxes
localization_loss_for_all_priors = localization_loss_for_all_priors * gt_boxes_map_offset_suppressed_with_pos_cond[:, :, 4]

total_pos_boxes = tf.reduce_sum(gt_boxes_map_offset_suppressed_with_pos_cond[:, :, 4:], axis=1)

In [35]:
localization_loss_for_all_priors

<tf.Tensor: shape=(2, 8732), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [36]:
localization_loss_for_all_priors / total_pos_boxes

<tf.Tensor: shape=(2, 8732), dtype=float32, numpy=
array([[nan, nan, nan, ..., nan, nan, nan],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)>

In [37]:
total_pos_boxes

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[ 0.],
       [52.]], dtype=float32)>

In [38]:
no_label_mask = tf.reduce_any( tf.not_equal(total_pos_boxes, tf.constant(0.)), axis = 1 )

In [39]:
tf.transpose([no_label_mask])

<tf.Tensor: shape=(2, 1), dtype=bool, numpy=
array([[False],
       [ True]])>

In [40]:
tf.where(tf.transpose([no_label_mask]), localization_loss_for_all_priors / total_pos_boxes, 
         tf.constant(0.))

<tf.Tensor: shape=(2, 8732), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [41]:
loss = SSDLoss()

In [42]:
actual_deltas = gt_boxes_map_offset_suppressed_with_pos_cond

huber_loss = tf.keras.losses.Huber(
    reduction=tf.keras.losses.Reduction.SUM
)
localization_loss_for_all_priors = huber_loss(actual_deltas[:, :, :4], pred_delta)

In [43]:
localization_loss_for_all_priors

<tf.Tensor: shape=(), dtype=float32, numpy=7444.3813>

In [44]:
loc_loss = loss.localization_loss(gt_boxes_map_offset_suppressed_with_pos_cond, 
                        pred_delta)

In [45]:
loc_loss

<tf.Tensor: shape=(2, 8732), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [48]:
# tf.where(no_label_mask, loc_loss / total_pos_boxes, tf.zeros(loc_loss.shape))

In [49]:
loss.confidence_loss(gt_labels_one_hot_encoded, pred_labels)

<tf.Tensor: shape=(2, 8732), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [50]:
categorical_cross_entropy = tf.losses.CategoricalCrossentropy(
            reduction=tf.losses.Reduction.NONE
        )

In [51]:
confidence_loss_for_all = categorical_cross_entropy(gt_labels_one_hot_encoded, pred_labels)

In [52]:
# gt_labels_one_hot_encoded
pos_cond = tf.reduce_any( tf.equal(gt_labels_one_hot_encoded[..., 1:], tf.constant(1.0)), axis = 2)
pos_mask = tf.cast(pos_cond, dtype=tf.float32)
total_pos_boxes = tf.reduce_sum(pos_mask, axis=1)

In [53]:
pos_loss = tf.reduce_sum(pos_mask * confidence_loss_for_all, axis=1)

In [54]:
pos_cond

<tf.Tensor: shape=(2, 8732), dtype=bool, numpy=
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])>

In [55]:
neg_cond = tf.reduce_any( tf.equal(gt_labels_one_hot_encoded[..., :1], tf.constant(1.0)), axis = 2)
neg_mask = tf.cast(neg_cond, dtype=tf.float32)

In [71]:
confidence_loss_for_all

<tf.Tensor: shape=(2, 8732), dtype=float32, numpy=
array([[2.9352515, 3.598501 , 3.8623304, ..., 5.561779 , 4.5376306,
        2.5427332],
       [2.9352515, 3.598501 , 3.8623304, ..., 5.561779 , 4.5376306,
        2.5427332]], dtype=float32)>

In [56]:
# set positive cases to -1
confidence_loss_for_all = tf.where(neg_cond, confidence_loss_for_all, 
                                   tf.negative(tf.ones(confidence_loss_for_all.shape))
                                  )

In [57]:
tf.reduce_sum(neg_mask, axis=1)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([8732., 8680.], dtype=float32)>

In [58]:
total_neg_bboxes = tf.cast(total_pos_boxes * 3, tf.int32)

In [59]:
total_neg_bboxes

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([  0, 156], dtype=int32)>

In [60]:
loss_sorted_indices = tf.argsort(confidence_loss_for_all, direction="DESCENDING")
loss_sorted_rank = tf.argsort(loss_sorted_indices)

In [61]:
loss_sorted_rank

<tf.Tensor: shape=(2, 8732), dtype=int32, numpy=
array([[4906, 2427, 1873, ...,  346,  909, 7448],
       [4881, 2413, 1862, ...,  344,  905, 7405]], dtype=int32)>

In [62]:
neg_cond = tf.less(loss_sorted_rank, tf.expand_dims(total_neg_bboxes, axis=1))
neg_mask = tf.cast(neg_cond, dtype=tf.float32)

In [63]:
neg_mask

<tf.Tensor: shape=(2, 8732), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [64]:
pos_loss

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([  0.     , 174.57834], dtype=float32)>

In [65]:
tf.reduce_sum(neg_mask * confidence_loss_for_all, axis=1)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([   0.    , 1144.7549], dtype=float32)>

In [66]:
conf_loss = tf.reduce_sum(neg_mask * confidence_loss_for_all, axis=1) + pos_loss

In [67]:
conf_loss

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([   0.    , 1319.3333], dtype=float32)>

In [68]:
pos_loss

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([  0.     , 174.57834], dtype=float32)>

In [69]:
tf.reduce_sum(neg_mask * confidence_loss_for_all, axis=1)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([   0.    , 1144.7549], dtype=float32)>

In [70]:
total_pos_boxes

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 0., 52.], dtype=float32)>